In [ ]:
%cd submodules/diff-gaussian-rasterization
!pip install .
%cd ../..
%cd submodules/simple-knn
!pip install .
%cd ../..

!pip install -r requirements.txt

In [ ]:
import numpy as np
import os
import time
from camera.camera import RealSenseCamera
from vision_slam import VisionSlam
from robot.flexiv import FlexivRobot

    
# Initialize the camera and robot
cam = RealSenseCamera()
cam_intrinsics = cam.getIntrinsics()  # Intrinsic parameters of the camera
camera_pose = np.load('config/camera_calib.npy')  # Extrinsic calibration matrix
flexiv_robot = FlexivRobot()

# Move robot to home position
flexiv_robot.move_to_home()
flexiv_robot.set_zero_ft()

# Generate trajectory for scanning
home_pose = flexiv_robot.get_tcp_pose(euler=True, degree=True)
object_position = home_pose[:3]  # Use only the XYZ component
vision_slam = VisionSlam()
trajectory = vision_slam.generate_3d_scan_trajectory(object_position)
# Create output directory
output_dir = "dataset"
os.makedirs(output_dir, exist_ok=True)

# Loop through each pose in the trajectory
for i, pose in enumerate(trajectory):
    flexiv_robot.cartesian_motion_control(pose)
    robot_pose = flexiv_robot.get_tcp_pose(matrix=True)  # 4x4 matrix
    time.sleep(1)
    color, depth = cam.get_data(hole_filling=False)
    
    rgb_path = os.path.join(output_dir, f"rgb_{i:04d}.npy")
    np.save(rgb_path, color)
    depth_path = os.path.join(output_dir, f"depth_{i:04d}.npy")
    np.save(depth_path, depth)
    pose_path = os.path.join(output_dir, f"pose_{i:04d}.npy")
    np.save(pose_path, robot_pose)
    




In [ ]:
from train import train_gs

train_gs()

